In [1]:
import pandas as pd

In [2]:
def segment_data(df):
    pos_dict = {}
    pos = ['PG','SG','PF','SF','C','F','G']
    
    for p in pos:
        pos_dict[p] = df.loc[df['POS'] == p]
        
    return pos_dict

In [3]:
def get_data(last_n_years,season):
    from time import strftime, localtime
    import pandas as pd
    
    this_year = int(strftime("%Y",localtime())) -1
    dfs = {}
    for y in range(this_year,this_year - last_n_years,-1):
        df = pd.read_csv("./player_stats/"+str(y)+"/"+season+".csv")
        dfs[y] = segment_data(df)
        
    return dfs

In [4]:
def player_slider():
    slider = dcc.Slider(
                        id = 'n_players',
                        min = 1,
                        max = 10,
                        value = 5,
                        marks = {str(n): str(n) for n in range(1,11)},
                        step = None,
                        tooltip={'always_visible':False}
                    )
    return slider

In [5]:
def year_slider():
    slider = dcc.Slider(
                        id = 'years',
                        min = min(reg_season.keys()),
                        max = max(reg_season.keys()),
                        value = max(reg_season.keys()),
                        marks = {year: str(year) for year in reg_season.keys()},
                        step = None,
                        tooltip={'always_visible':False}
                    )
    return slider

In [6]:
def player_stats():
    stats = dcc.RadioItems(
                            options=[
                                {'label': 'PTS','value':'PTS'},
                                {'label': 'AST','value':'AST'},
                                {'label': 'REB','value':'REB'},
                                {'label': 'BLK','value':'BLK'},
                                {'label': 'STL','value':'STL'}
                            ],
                            value = 'PTS',
                            labelStyle={'display': 'inline-block'},
                            id = "stats",
                            style = {'color':colours['radio_button'],
                                     'fontSize':text_size['radio_button']}
                    )
    return stats

In [7]:
def pos_dropdown():
    d = dcc.Dropdown(
                    options = [
                        {"value":"PG", "label":"Point Guard"},
                        {"value":"SG", "label":"Shooting Guard"},
                        {"value":"G", "label":"Guard"},
                        {"value":"PF", "label":"Power Forward"},
                        {"value":"SF", "label":"Small Forward"},
                        {"value":"F", "label":"Forward"},
                        {"value":"C", "label":"Centre"}
                    ],
                    value = "PG",
                    id = "pos_drop"
                )
    return d

In [8]:
def season_dropdown():
    d = dcc.Dropdown(
                    options = [
                        {"value":"regular", "label":"Regular"},
                        {"value":"post", "label":"Post"}
                    ],
                    value = "regular",
                    id = "season_drop"
                )
    return d

In [9]:
from jupyter_dash import JupyterDash
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
from datetime import date
from dash import dcc

season_types = ["regular_season","post_season"]
r_season = season_types[0]
p_season = season_types[1]


y = 2021
pos = 'PG'
top_n_players = 5

reg_season = get_data(5,r_season)
post_season = get_data(5,p_season)

seasons = {"regular":reg_season, "post":post_season}

sorted_df = seasons["regular"][y][pos].sort_values("PTS",ascending = False).head(top_n_players)

fig = px.bar(sorted_df, x='PTS', y="Name", color="Team", barmode='overlay',opacity=1)
fig.update_layout(title_text="PTS for top "+str(top_n_players)+" players at "+ pos +\
                  " in the "+str(y) +" regular season", title_x=0.5)




app = JupyterDash(__name__)
colours = {'text': '#7FDBFF', 'background':'#333333','radio_button':'#BBBBBB'}
text_size = {'H1':48,'H2':40,'text':28,'radio_button':20}

app.layout = html.Div(style={'backgroundColor':colours['background'],'fontFamily':'Arial'}, children=[

    html.H1(children='NBA Data visualisation',
        style = {'textAlign': 'center',
                 'color':colours['text'],
                 'fontSize':text_size['H1']}),


    html.Div(children=[
        
            html.Div([html.Label("Position: ", style = {'textAlign': 'center',
                         'color':colours['text'],
                         'fontSize':text_size['text']}),
                        pos_dropdown()
                    ],
                style = {'textAlign': 'left',"flex":1}
            ),
        
            html.Div([html.Label("Season: ", style = {'textAlign': 'center',
                         'color':colours['text'],
                         'fontSize':text_size['text']}),
                        season_dropdown()
                     ],
                style = {'textAlign': 'center',"flex":1}
            ),
        
        
            html.Div([html.Label("Stats", style = {'textAlign': 'center',
                         'color':colours['text'],
                         'fontSize':text_size['text']}),
                        player_stats()
                     ],
                style = {'textAlign': 'right',"flex":1}
            )
        
        
            

        ],style = {'display':'flex','flex-direction': 'row'}
    ),
    
    player_slider(),
    
    dcc.Graph(figure = fig, id = 'graph'),
    
    year_slider()
])

@app.callback(
    Output('graph','figure'),
    Input('stats','value'),
    Input('years','value'),
    Input('pos_drop','value'),
    Input('n_players','value'),
    Input('season_drop','value'))
def update_figure(stat,y,pos,top_n_players,season_type):
    
    sorted_df = seasons[season_type][y][pos].sort_values(stat,ascending = False).head(top_n_players)
    fig = px.bar(sorted_df, x=stat, y='Name', color="Team", barmode='overlay', opacity=1)
    
    fig.update_layout(yaxis={'categoryorder':'max ascending'},
                      title_text= stat+ " for top "+str(top_n_players)+" players at "+ pos +\
                      " in the "+str(y) +" regular season", title_x=0.5)

    return fig

In [10]:
app.run_server(mode = "external")

Dash app running on http://127.0.0.1:8050/


In [11]:
seasons['regular']

{2021: {'PG':                  Name POS  FGM   FGA   FG%  3PM   3PA   3P%  FTM  FTA  ...  \
  2        Kemba Walker  PG  6.6  15.7  42.0  3.0   8.2  36.0  3.1  3.5  ...   
  3        Marcus Smart  PG  4.2  10.6  39.8  1.9   5.9  33.0  2.7  3.4  ...   
  7    Payton Pritchard  PG  2.8   6.3  44.0  1.5   3.8  41.1  0.6  0.7  ...   
  9         Jeff Teague  PG  2.3   5.5  41.5  0.8   1.6  46.4  1.5  1.8  ...   
  16     Carsen Edwards  PG  1.5   3.6  42.3  0.6   2.0  28.6  0.4  0.4  ...   
  ..                ...  ..  ...   ...   ...  ...   ...   ...  ...  ...  ...   
  577       George Hill  PG  4.4   8.6  50.8  1.6   4.1  38.6  1.5  1.8  ...   
  580      Theo Maledon  PG  3.5   9.4  36.8  1.6   4.8  33.5  1.6  2.1  ...   
  594    Damian Lillard  PG  9.0  19.9  45.1  4.1  10.5  39.1  6.7  7.2  ...   
  612   Jordan Clarkson  PG  6.7  15.8  42.5  3.1   8.8  34.7  1.9  2.1  ...   
  614       Mike Conley  PG  5.6  12.5  44.4  2.7   6.6  41.2  2.4  2.8  ...   
  
        DR  REB  AST  STL